In [125]:
#SPLITTING THE SOUNDS 

In [163]:
import pandas as pd 

sound_final_df=pd.read_csv('E:/FEATURE/final_csv.csv')
sound_final_df.head()
sound_final_df['end']=6
sound_final_df['start']=0
sound_final_df.head()

,Unnamed: 0,start,end,crackles,wheezes,file_name,Patient_number,acquisition_mode,Normal,Status,Condition
0,0,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
1,1,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
2,2,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
3,3,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
4,4,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI


In [164]:
# slicing the given lun sound samples

#create new column to store length of a slice / slice_length = end - start
sound_final_df['length_of_slice']=sound_final_df['end'].sub(sound_final_df['start'],axis = 0)
maximum_length_of_slice=max(sound_final_df['length_of_slice'])
maximum_length_of_slice

6

In [165]:
sound_final_df.head()

,Unnamed: 0,start,end,crackles,wheezes,file_name,Patient_number,acquisition_mode,Normal,Status,Condition,length_of_slice
0,0,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI,6
1,1,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI,6
2,2,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI,6
3,3,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI,6
4,4,0,6,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI,6


In [166]:
def slice_data(start, end, raw_data,  sample_rate):
    max_ind = len(raw_data) 
    start_ind = min(int(start * sample_rate), max_ind)
    end_ind = min(int(end * sample_rate), max_ind)
    return raw_data[start_ind: end_ind]

In [167]:
import math

import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats
time=math.ceil(boxplot_stats(sound_final_df['length_of_slice'])[0]['whishi'])/6

In [168]:
time


1.0

In [169]:
import librosa 
import IPython.display as ipd
import librosa
import librosa.display

In [170]:
#calculate the slice length of all sound samples


def calculate_length(samp_rate=22050, time=time, acquisition_mode=0):
   
    cal_length= 0
    #to calculate mono acacquisition_mode=1
    if acquisition_mode == 1:
        cal_length = samp_rate * time
     
    #to calculate stereo acacquisition_mode=0   
    else:
        cal_length =(samp_rate * time)*2

    return cal_length

In [171]:
#create output path
import os
os.makedirs('output')

In [172]:
import pandas as pd
df_path=('E:/FEATURE/final_csv.csv')
df=pd.read_csv(df_path)
df.head()

,Unnamed: 0,start,end,crackles,wheezes,file_name,Patient_number,acquisition_mode,Normal,Status,Condition
0,0,0.036,0.579,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
1,1,0.579,2.450,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
2,2,2.450,3.893,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
3,3,3.893,5.793,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI
4,4,5.793,7.521,0,0,101_1b1_Al_sc_Meditron,101.0,sc,1,Normal,URTI


In [173]:
folders = df['Status'].unique()
folders

array(['Normal', 'wheezes', 'crackles'], dtype=object)

In [174]:
import os
for Conditions in folders:
    path = os.path.join('output', Conditions)
    os.makedirs(path)

In [175]:
# importng text files
import os
# save all text files names to files_text variable in the data set and print 
audio_text_path='E:/EDA/audio_and_txt_files'
files_text=[text.split('.')[0] for text in os.listdir(path=audio_text_path) if '.txt' in text]

In [176]:
All_Text_file=[]
for f in files_text:
    df1=pd.read_csv(audio_text_path + '/' + f +'.txt', sep='\t',names=['start','end','crackles','wheezes'])
    df1['file_name']=f

In [177]:
import soundfile as sf

In [178]:
#sliced audio save in separated floders 
audio_text_path='E:/EDA/audio_and_txt_files'
i=0
for idx, row in sound_final_df.iterrows():
    filename = row['file_name']
    start = row['start']
    end = row['end']
    Condition = row['Status']
    
    
    #check len and force to 6 sec if more than that
    if time <= 2:
        end = start + time
    
        aud_loc = audio_text_path + '/' + f + '.wav'
    
    if idx != 0:
        if sound_final_df.iloc[idx-1]['file_name'] == filename:
            i=i+1
        else:
            i=0
    n_filename = filename + '_' + str(i) + '.wav'
    path = 'output' +'/'+ Condition + '/' + n_filename
    print(path)
    

    data, samplingrate = librosa.load(aud_loc)
    sliced_data = slice_data(start=start, end=end, raw_data=data, sample_rate=samplingrate)
    
    a_len = calculate_length(samp_rate=samplingrate, acquisition_mode=row['acquisition_mode']=='sc')
    
    padded_data = librosa.util.pad_center(sliced_data, a_len)

    sf.write(file=path, data=padded_data, samplerate=samplingrate)

output/Normal/101_1b1_Al_sc_Meditron_0.wav


C:\Users\DELL\AppData\Local\Temp/ipykernel_3804/2406455482.py:32: FutureWarning: Pass size=22050.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  padded_data = librosa.util.pad_center(sliced_data, a_len)


output/Normal/101_1b1_Al_sc_Meditron_1.wav
output/Normal/101_1b1_Al_sc_Meditron_2.wav
output/Normal/101_1b1_Al_sc_Meditron_3.wav
output/Normal/101_1b1_Al_sc_Meditron_4.wav
output/Normal/101_1b1_Al_sc_Meditron_5.wav
output/Normal/101_1b1_Al_sc_Meditron_6.wav
output/Normal/101_1b1_Al_sc_Meditron_7.wav
output/Normal/101_1b1_Al_sc_Meditron_8.wav
output/Normal/101_1b1_Al_sc_Meditron_9.wav
output/Normal/101_1b1_Al_sc_Meditron_10.wav
output/Normal/101_1b1_Al_sc_Meditron_11.wav
output/Normal/101_1b1_Pr_sc_Meditron_0.wav
output/Normal/101_1b1_Pr_sc_Meditron_1.wav
output/Normal/101_1b1_Pr_sc_Meditron_2.wav
output/Normal/101_1b1_Pr_sc_Meditron_3.wav
output/Normal/101_1b1_Pr_sc_Meditron_4.wav
output/Normal/101_1b1_Pr_sc_Meditron_5.wav
output/Normal/101_1b1_Pr_sc_Meditron_6.wav
output/Normal/101_1b1_Pr_sc_Meditron_7.wav
output/Normal/101_1b1_Pr_sc_Meditron_8.wav
output/Normal/101_1b1_Pr_sc_Meditron_9.wav
output/Normal/101_1b1_Pr_sc_Meditron_10.wav
output/Normal/102_1b1_Ar_sc_Meditron_0.wav
output/N

C:\Users\DELL\AppData\Local\Temp/ipykernel_3804/2406455482.py:32: FutureWarning: Pass size=44100.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  padded_data = librosa.util.pad_center(sliced_data, a_len)


output/Normal/103_2b2_Ar_mc_LittC2SE_1.wav
output/wheezes/103_2b2_Ar_mc_LittC2SE_2.wav
output/wheezes/103_2b2_Ar_mc_LittC2SE_3.wav
output/wheezes/103_2b2_Ar_mc_LittC2SE_4.wav
output/Normal/103_2b2_Ar_mc_LittC2SE_5.wav
output/Normal/104_1b1_Al_sc_Litt3200_0.wav
output/Normal/104_1b1_Al_sc_Litt3200_1.wav
output/Normal/104_1b1_Al_sc_Litt3200_2.wav
output/Normal/104_1b1_Al_sc_Litt3200_3.wav
output/Normal/104_1b1_Al_sc_Litt3200_4.wav
output/Normal/104_1b1_Al_sc_Litt3200_5.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_0.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_1.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_2.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_3.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_4.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_5.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_6.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_7.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_8.wav
output/wheezes/104_1b1_Ar_sc_Litt3200_9.wav
output/Normal/104_1b1_Ar_sc_Litt3200_10.wav
output/Normal/104_1b1_Ar_sc_Litt3200_11.

...

output/wheezes/206_1b1_Pl_sc_Meditron_4.wav
output/wheezes/206_1b1_Pl_sc_Meditron_5.wav
output/wheezes/206_1b1_Pl_sc_Meditron_6.wav
output/wheezes/206_1b1_Pl_sc_Meditron_7.wav
output/Normal/207_2b2_Al_mc_AKGC417L_0.wav
output/Normal/207_2b2_Al_mc_AKGC417L_1.wav
output/Normal/207_2b2_Al_mc_AKGC417L_2.wav
output/Normal/207_2b2_Al_mc_AKGC417L_3.wav
output/Normal/207_2b2_Al_mc_AKGC417L_4.wav
output/Normal/207_2b2_Al_mc_AKGC417L_5.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_0.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_1.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_2.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_3.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_4.wav
output/Normal/207_2b2_Ar_mc_AKGC417L_5.wav
output/Normal/207_2b2_Pl_mc_AKGC417L_0.wav
output/crackles/207_2b2_Pl_mc_AKGC417L_1.wav
output/Normal/207_2b2_Pl_mc_AKGC417L_2.wav
output/crackles/207_2b2_Pl_mc_AKGC417L_3.wav
output/Normal/207_2b2_Pl_mc_AKGC417L_4.wav
output/Normal/207_2b2_Pl_mc_AKGC417L_5.wav
output/Normal/207_2b2_Pr_mc_AKGC417L_0.wav
out

output/Normal/210_1b1_Ar_sc_Meditron_12.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_0.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_1.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_2.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_3.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_4.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_5.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_6.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_7.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_8.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_9.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_10.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_11.wav
output/Normal/211_1p2_Ar_mc_AKGC417L_12.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_0.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_1.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_2.wav
output/crackles/211_1p2_Pl_mc_AKGC417L_3.wav
output/crackles/211_1p2_Pl_mc_AKGC417L_4.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_5.wav
output/crackles/211_1p2_Pl_mc_AKGC417L_6.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_7.wav
output/wheezes/211_1p2_Pl_mc_AKGC417L_8

output/Normal/213_1p5_Ar_mc_AKGC417L_0.wav
output/Normal/213_1p5_Ar_mc_AKGC417L_1.wav
output/Normal/213_1p5_Ar_mc_AKGC417L_2.wav
output/Normal/213_1p5_Ar_mc_AKGC417L_3.wav
output/Normal/213_1p5_Ar_mc_AKGC417L_4.wav
output/Normal/213_1p5_Ar_mc_AKGC417L_5.wav
output/Normal/213_1p5_Pl_mc_AKGC417L_0.wav
output/crackles/213_1p5_Pl_mc_AKGC417L_1.wav
output/Normal/213_1p5_Pl_mc_AKGC417L_2.wav
output/Normal/213_1p5_Pl_mc_AKGC417L_3.wav
output/Normal/213_1p5_Pl_mc_AKGC417L_4.wav
output/Normal/213_1p5_Pl_mc_AKGC417L_5.wav
output/Normal/213_1p5_Pr_mc_AKGC417L_0.wav
output/crackles/213_1p5_Pr_mc_AKGC417L_1.wav
output/Normal/213_1p5_Pr_mc_AKGC417L_2.wav
output/Normal/213_1p5_Pr_mc_AKGC417L_3.wav
output/Normal/213_1p5_Pr_mc_AKGC417L_4.wav
output/crackles/213_1p5_Pr_mc_AKGC417L_5.wav
output/Normal/213_1p5_Tc_mc_AKGC417L_0.wav
output/Normal/213_1p5_Tc_mc_AKGC417L_1.wav
output/Normal/213_1p5_Tc_mc_AKGC417L_2.wav
output/Normal/213_1p5_Tc_mc_AKGC417L_3.wav
output/Normal/213_1p5_Tc_mc_AKGC417L_4.wav
outpu

output/wheezes/218_1b1_Pr_sc_Meditron_9.wav
output/wheezes/218_1b1_Pr_sc_Meditron_10.wav
output/wheezes/218_1b1_Pr_sc_Meditron_11.wav
output/wheezes/218_1b1_Pr_sc_Meditron_12.wav
output/wheezes/218_1b1_Pr_sc_Meditron_13.wav
output/wheezes/218_1b1_Pr_sc_Meditron_14.wav
output/wheezes/218_1b1_Pr_sc_Meditron_15.wav
output/Normal/218_1b1_Pr_sc_Meditron_16.wav
output/Normal/218_1b1_Pr_sc_Meditron_17.wav
output/Normal/218_1p1_Ar_sc_Litt3200_0.wav
output/Normal/218_1p1_Ar_sc_Litt3200_1.wav
output/Normal/218_1p1_Ar_sc_Litt3200_2.wav
output/wheezes/218_1p1_Ar_sc_Litt3200_3.wav
output/wheezes/218_1p1_Ar_sc_Litt3200_4.wav
output/wheezes/218_1p1_Ar_sc_Litt3200_5.wav
output/Normal/218_1p1_Pl_sc_Litt3200_0.wav
output/crackles/218_1p1_Pl_sc_Litt3200_1.wav
output/crackles/218_1p1_Pl_sc_Litt3200_2.wav
output/Normal/218_1p1_Pl_sc_Litt3200_3.wav
output/wheezes/218_1p1_Pl_sc_Litt3200_4.wav
output/wheezes/218_1p1_Pl_sc_Litt3200_5.wav
output/wheezes/218_1p1_Pl_sc_Litt3200_6.wav
output/wheezes/218_1p1_Pl_sc_

output/wheezes/223_1b1_Ar_sc_Meditron_0.wav
output/wheezes/223_1b1_Ar_sc_Meditron_1.wav
output/wheezes/223_1b1_Ar_sc_Meditron_2.wav
output/wheezes/223_1b1_Ar_sc_Meditron_3.wav
output/wheezes/223_1b1_Ar_sc_Meditron_4.wav
output/wheezes/223_1b1_Ar_sc_Meditron_5.wav
output/wheezes/223_1b1_Ar_sc_Meditron_6.wav
output/wheezes/223_1b1_Ar_sc_Meditron_7.wav
output/wheezes/223_1b1_Ar_sc_Meditron_8.wav
output/wheezes/223_1b1_Ar_sc_Meditron_9.wav
output/crackles/223_1b1_Ll_sc_Meditron_0.wav
output/Normal/223_1b1_Ll_sc_Meditron_1.wav
output/crackles/223_1b1_Ll_sc_Meditron_2.wav
output/crackles/223_1b1_Ll_sc_Meditron_3.wav
output/crackles/223_1b1_Ll_sc_Meditron_4.wav
output/crackles/223_1b1_Ll_sc_Meditron_5.wav
output/crackles/223_1b1_Ll_sc_Meditron_6.wav
output/crackles/223_1b1_Ll_sc_Meditron_7.wav
output/crackles/223_1b1_Ll_sc_Meditron_8.wav
output/crackles/223_1b1_Ll_sc_Meditron_9.wav
output/crackles/223_1b1_Lr_sc_Meditron_0.wav
output/crackles/223_1b1_Lr_sc_Meditron_1.wav
output/crackles/223_1b